In [31]:
import numpy as np
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import glob
import cv2
import torch
from sklearn.cluster import KMeans
import os

In [32]:
import json

def read_poses_from_json(file_path):
    """Reads the pose data from a JSON file."""
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def write_matrices_to_txt(data, output_file_path):
    """Writes the 4x4 pose matrices to a text file."""
    with open(output_file_path, 'w') as file:
        for frame_id, frame_data in data.items():
            pose = frame_data['pose']
            #file.write(f"{frame_id}:\n")
            for row in pose:
                formatted_row = ' '.join(f"{value:.18e}" for value in row)
                file.write(formatted_row + "\n")

# Paths to your JSON input and text output files
input_json_path = '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/pose_intrinsic_imu.json'  # Change this to the path of your JSON file
output_txt_path = '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/processedData/traj.txt'  # Change this to your desired output file path

# Read the data from JSON
pose_data = read_poses_from_json(input_json_path)

# Write the matrices to a text file
write_matrices_to_txt(pose_data, output_txt_path)


In [35]:
import torch.nn.functional as F
def readDepth(filepath):
    depth = cv2.imread(filepath, cv2.IMREAD_UNCHANGED)
    depth_data = depth.astype(np.float32) / 6553.5
    depth_data = torch.from_numpy(depth_data)*10
    #print(depth_data.shape)
    #print(depth_data[0,0])
    return depth_data


In [36]:
import glob
from tqdm import tqdm  # Import tqdm
def save_depth_image(input_path, output_filepath):
    """Saves the depth image to a file."""
    depth_paths = sorted(
            glob.glob(os.path.join(input_path, "*.png"))
        )
    #print(depth_paths)
    for i, depth_path in enumerate(tqdm(depth_paths, desc='Upsampling depth images')):
        depth_data=readDepth(depth_path)
        upsampled_depth_bilinear = F.interpolate(depth_data.unsqueeze(0).unsqueeze(0), 
                                                scale_factor=7.5, 
                                                mode='bilinear', 
                                                align_corners=True).squeeze()
        #print(upsampled_depth_bilinear.shape)
        upsampled_depth = upsampled_depth_bilinear * 6553.5
        upsampled_depth = upsampled_depth.numpy().astype(np.uint16)

        cv2.imwrite(os.path.join(output_filepath, f"frame_{str(i).zfill(6)}.png"), upsampled_depth)
save_depth_image('/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/depth/', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/processedData/depth')

Upsampling depth images:   0%|          | 0/9775 [00:00<?, ?it/s]

Upsampling depth images: 100%|██████████| 9775/9775 [17:18<00:00,  9.42it/s]
